### Capston project - Netflix movies - model training

In [1]:
# import libraries
import sys
import re
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import pickle

nltk.download(['punkt', 'wordnet', 'omw-1.4'])

# define regex characters
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tomaslorincfpt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tomaslorincfpt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/tomaslorincfpt/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# define tokenize function used for text transformation
def tokenize(text):
    """
    Function which tokenize message using regular expressions
    :param text: String containing message
    :return: clean_tokens: list of words containing tokenized and cleaned message
    """
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

# define function which displays results of model training
def display_results(cv, y_test, y_pred, labels):
    """
    This function visualise trained model
    :param cv: Any
    :param y_test: Any
    :param y_pred: Any
    :param labels: Any
    :return: none
    """
    accuracy = (y_pred == y_test).mean()
    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

### First we need to load data from SQL DB

In [3]:
# load data
engine = create_engine('sqlite:///app/moviesdata.db')
df = pd.read_sql_table(table_name='name', con=engine)
X = df['title']
Y = df[df.columns].drop(['title'], axis = 1)

Y = Y.astype(int)
categories = Y.columns

### Then we need to split them

In [4]:
# split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### and create model using pipelines

In [5]:
# build model
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('Tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])
parameters = {'clf__estimator__n_estimators': [10]}

model = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=1)

### Now we are able to fit our model

In [6]:
# fit model
model.fit(X_train, Y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ...clf__estimator__n_estimators=10;, score=0.065 total time=   1.9s
[CV 2/5] END ...clf__estimator__n_estimators=10;, score=0.072 total time=   1.3s
[CV 3/5] END ...clf__estimator__n_estimators=10;, score=0.065 total time=   1.2s
[CV 4/5] END ...clf__estimator__n_estimators=10;, score=0.076 total time=   1.2s
[CV 5/5] END ...clf__estimator__n_estimators=10;, score=0.062 total time=   1.2s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x16622e8b0>)),
                                       ('Tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=1, param_grid={'clf__estimator__n_estimators': [10]},
             verbose=3)

### Analyse results

as we can see our results are prety good. We have nice accuracy in all geners expect drama which could be caused just because almost every movie has little bit of drama which makes this category less accurate. 

In [7]:
# display results
y_pred = model.predict(X_test)
display_results(model, Y_test, y_pred, categories)

Labels: Index(['drama', 'fantasy', 'war', 'comedy', 'thriller', 'crime', 'romance',
       'action', 'western', 'history', 'documentation', 'music', 'horror',
       'scifi', 'animation', 'family', 'reality', 'sport', 'european'],
      dtype='object')
Accuracy: drama            0.621832
fantasy          0.889864
war              0.971735
comedy           0.615984
thriller         0.763158
crime            0.811891
romance          0.814815
action           0.766082
western          0.992203
history          0.962963
documentation    0.834308
music            0.951267
horror           0.926901
scifi            0.897661
animation        0.908382
family           0.875244
reality          0.956140
sport            0.961988
european         0.894737
dtype: float64

Best Parameters: {'clf__estimator__n_estimators': 10}


### Final step - save model into pickle file

In [8]:
#save model
with open('app/netflix_model.pkl', 'wb') as f:
    pickle.dump(model, f)
pass